In [1]:
import pandas as pd
import json
import os

In [2]:
out_dir = "./data/out"
in_dir = "./data/in"

In [3]:
out_dir = "./data/out"
in_dir = "./data/in"

In [4]:
experiment_name = "70-xrp"
optimizer_name = "brute force"
optimizer_dir = os.path.join(out_dir, optimizer_name)

experiment_names = [
    "62-ada",
    "63-btc",
    "64-dash",
    "65-doge",
    "66-eth",
    "67-ltc",
    "68-sol",
    "69-xlm",
    "70-xrp"
]

In [24]:
quote_asset_name = "USDT"

pair_col_name = "Obchodní pár"
indic_col_name = "Ukazatel"
period_col_name = "Perioda"
levels_col_name = "Nákupní úrovně (%)"
sizes_col_name = "Nákupní velikosti (%)"
config_overview = pd.DataFrame(columns = [pair_col_name, indic_col_name, period_col_name, levels_col_name,
                                          sizes_col_name])


net_profit_col_name = "Zisk ({})".format(quote_asset_name)
profit_factor_col_name = "Ziskový faktor"
win_count_col_name = "Počet ziskových pozic"
loss_count_col_name = "Počet ztrátových pozic"
max_close_balance_drawdown_col_name = "Největší sestupný pohyb zůstatku po uzavření pozice (%)"
max_equity_drawdown_col_name = "Největší sestupný pohyb čisté hodnoty (%)"
stats_overview = pd.DataFrame(columns = [pair_col_name, net_profit_col_name, profit_factor_col_name,
                                         win_count_col_name, loss_count_col_name,
                                         max_close_balance_drawdown_col_name, max_equity_drawdown_col_name])

In [25]:
def ftos(num):
    return "{:.1f}".format(num)

In [26]:
def rtof(frac):
    num, denom = frac.split("/")
    return float(num)/float(denom)

In [27]:
def convert_levels(levels):
    out_levels = []
    
    for level in levels:
        out_levels.append(ftos((rtof(level)-1.)*100))
        
    return out_levels

In [28]:
def convert_sizes(sizes):
    out_sizes = []
    
    for size in sizes:
        out_sizes.append(ftos(rtof(size)*100))
        
    return out_sizes

In [29]:
for experiment_name in experiment_names:
    experiment_dir = os.path.join(optimizer_dir, experiment_name)
    settings_path = os.path.join(experiment_dir, "settings.json")
    best_states_path = os.path.join(experiment_dir, "best-states.json")
    
    settings_file = open(settings_path, "r")
    settings_doc = json.loads(settings_file.read())
    
    pair_doc = settings_doc["candles"]["currency pair"];
    base_name = pair_doc["base"]
    quote_name = pair_doc["quote"]
    
    pair_name = "{}/{}".format(base_name, quote_name)
    
    best_states_file = open(best_states_path, "r")
    best_states_doc = json.loads(best_states_file.read())
    best_state_doc = best_states_doc[0]
    
    best_config = best_state_doc["configuration"]
    indic_doc = best_config["indicator"]
    indic_name = indic_doc["type"].upper()
    indic_period = indic_doc["period"]
    levels = convert_levels(best_config["levels"])
    sizes = convert_sizes(best_config["open sizes"])
    
    config_overview.loc[len(config_overview)] = {
        pair_col_name : pair_name,
        indic_col_name : indic_name,
        period_col_name : indic_period,
        levels_col_name : levels,
        sizes_col_name : sizes,
    }
    
    best_stats = best_state_doc["statistics"]
    net_profit = best_stats["net profit"]
    profit_factor = best_stats["profit factor"]
    win_count = best_stats["win count"]
    loss_count = best_stats["loss count"]
    max_close_balance_drawdown = ftos(best_stats["close balance"]["max drawdown"]["percent"])
    max_equity_drawdown = ftos(best_stats["equity"]["max drawdown"]["percent"])
    
    stats_overview.loc[len(stats_overview)] = {
        pair_col_name : pair_name,
        net_profit_col_name : net_profit,
        profit_factor_col_name : profit_factor,
        win_count_col_name : win_count,
        loss_count_col_name : loss_count,
        max_close_balance_drawdown_col_name : max_close_balance_drawdown,
        max_equity_drawdown_col_name : max_equity_drawdown,
    }

In [30]:
config_overview

,Obchodní pár,Ukazatel,Perioda,Nákupní úrovně (%),Nákupní velikosti (%)
0,ADA/USDT,SMA,84,"[-17.9, -20.2, -21.4]","[12.5, 12.5, 75.0]"
1,BTC/USDT,SMA,15,"[-8.3, -9.5, -10.7]","[12.5, 12.5, 75.0]"
2,DASH/USDT,SMA,15,"[-14.3, -17.9, -19.0]","[12.5, 75.0, 12.5]"
3,DOGE/USDT,SMA,15,"[-11.9, -20.2, -22.6]","[12.5, 12.5, 75.0]"
4,ETH/USDT,SMA,9,"[-11.9, -13.1, -16.7]","[75.0, 12.5, 12.5]"
5,LTC/USDT,SMA,12,"[-10.7, -13.1, -14.3]","[12.5, 12.5, 75.0]"
6,SOL/USDT,SMA,3,"[-8.3, -13.1, -20.2]","[12.5, 62.5, 25.0]"
7,XLM/USDT,SMA,60,"[-11.9, -22.6, -23.8]","[12.5, 12.5, 75.0]"
8,XRP/USDT,SMA,9,"[-10.7, -11.9, -21.4]","[12.5, 50.0, 37.5]"


In [31]:
stats_overview

,Obchodní pár,Zisk (USDT),Ziskový faktor,Počet ziskových pozic,Počet ztrátových pozic,Největší sestupný pohyb zůstatku po uzavření pozice (%),Největší sestupný pohyb čisté hodnoty (%)
0,ADA/USDT,80175.218750,28.827568,26,3,-17.1,-26.4
1,BTC/USDT,48372.164062,4.678106,67,16,-28.1,-47.5
2,DASH/USDT,19288.505859,20.072048,23,4,-8.9,-40.0
3,DOGE/USDT,147337.359375,86.228378,59,6,-2.0,-22.2
4,ETH/USDT,80965.843750,22.824848,46,4,-8.6,-29.3
5,LTC/USDT,85502.367188,5.691393,64,13,-9.6,-38.5
6,SOL/USDT,17851.925781,4.825638,74,18,-11.1,-20.2
7,XLM/USDT,78155.039062,139.264679,52,10,-0.3,-42.5
8,XRP/USDT,32632.691406,3.751921,59,10,-22.5,-33.4


In [32]:
config_overview.to_latex(index=False)

/var/folders/yp/jr0t3y992l3_0q5yzdp72qdw0000gn/T/ipykernel_26730/3842703109.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  config_overview.to_latex(index=False)


'\\begin{tabular}{llrll}\n\\toprule\nObchodní pár & Ukazatel &  Perioda &    Nákupní úrovně (\\%) & Nákupní velikosti (\\%) \\\\\n\\midrule\n    ADA/USDT &      SMA &       84 & [-17.9, -20.2, -21.4] &    [12.5, 12.5, 75.0] \\\\\n    BTC/USDT &      SMA &       15 &   [-8.3, -9.5, -10.7] &    [12.5, 12.5, 75.0] \\\\\n   DASH/USDT &      SMA &       15 & [-14.3, -17.9, -19.0] &    [12.5, 75.0, 12.5] \\\\\n   DOGE/USDT &      SMA &       15 & [-11.9, -20.2, -22.6] &    [12.5, 12.5, 75.0] \\\\\n    ETH/USDT &      SMA &        9 & [-11.9, -13.1, -16.7] &    [75.0, 12.5, 12.5] \\\\\n    LTC/USDT &      SMA &       12 & [-10.7, -13.1, -14.3] &    [12.5, 12.5, 75.0] \\\\\n    SOL/USDT &      SMA &        3 &  [-8.3, -13.1, -20.2] &    [12.5, 62.5, 25.0] \\\\\n    XLM/USDT &      SMA &       60 & [-11.9, -22.6, -23.8] &    [12.5, 12.5, 75.0] \\\\\n    XRP/USDT &      SMA &        9 & [-10.7, -11.9, -21.4] &    [12.5, 50.0, 37.5] \\\\\n\\bottomrule\n\\end{tabular}\n'